# PRÁCTICA GUIADA: Test Training Workflow

## Introducción

El objetivo de esta práctica es recorrer y profundizar las diferentes partes de la etapa de "Modelado" en el flujo de trabajo de Data Science. En efecto, supongamos que ya hemos concluido la etapa de limpieza y preprocesamiento de los datos. El paso siguiente es comenzar la etapa de análisis y modelado de la información. Hay tres grandes tareas que serán habitualmente realizadas en esta etapa (no excluyentes entre sí, de hecho será habitual encarar las tres en simultáneo):

* Realizar el proceso de tunning de los hiperparámetros
* Evaluar la performance de un modelo
* Evaluar la performance de varios modelos

A continuación presentamos un esquema estilizado del proceso:

#### Un esquema posible del proceso de estimación y modelado
_______________________________________________________________________________________________________________

1. Realizar una separación entre Train y Test Sets.

2. Sobre el **set de entrenamiento (Train Set):** 
    
    1. *Selección de los modelos a evaluar:* por ahora estamos trabajando sobre el de regresión modelo lineal y sus extensiones.
    
    2. *Proceso de Tunning de los Hiperparámetros:* hasta ahora hemos visto como primer hiperparámetro el $\alpha$ para los modelos de regresión regularizados. Este proceso se realiza generalmente a partir de una estrategia de cross-validation.
    
    3. *Entrenamiento del modelo final:* Luego del proceso de selección y tunning de los hiperparámetros, es necesario realizar el entrenamiento final del modelo. Para ello, estimamos un modelo sobre el total del Training Set.
    
3. Sobre el **set de testing (Test Set):**
       
    1. Validación final del modelo (cálculo de errores y scores).
      
________

En este LAB trabajaremos sobre estas diferentes etapas a partir de un dataset que busca modelar la progresión de la enfermedad diabetes. En efecto, el dataset contiene 10 variables fisiológicas (edad. sexo, índice de masa corporal y seis mediciones del plasma sanguíneo) medidas en 442 pacientes y un indicador de la progresión de la enfermedad luego de un año base.

La idea será hacer predicciones sobre este indicador de progresión de la enfermedad, identificando los mejores modelos y los mejores predictores.

El dataset está extraido de [un paper de Hastie y Tibshiriani](https://web.stanford.edu/~hastie/Papers/LARS/LeastAngle_2002.pdf)

Para ello, recorreremos los diferentes pasos de esta etapa.

* En primer lugar, como siempre, importamos los diferentes paquetes a utilizar.

In [2]:
% matplotlib inline

from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = 10, 10

import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
from sklearn import datasets
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

* Luego, cargamos el dataset y definimos los nombres de colmnas

In [6]:
columns = "age sex bmi map tc ldl hdl tch ltg glu".split()
diabetes = datasets.load_diabetes()
df = pd.DataFrame(diabetes.data, columns=columns)
y = diabetes.target  # progresion de la enfermedad

df.head()

,age,sex,bmi,map,tc,ldl,hdl,tch,ltg,glu
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641


* Pareciera que las variables ya están normalizadas: 
    + $\bar{x_{j}} \approx 0$
    
    + $s_{j}=0.047619$

In [7]:
df.apply(['mean','std']).T

,mean,std
age,-3.634285e-16,0.047619
sex,1.308343e-16,0.047619
bmi,-8.045349e-16,0.047619
map,1.281655e-16,0.047619
tc,-8.835316e-17,0.047619
ldl,1.327024e-16,0.047619
hdl,-4.574646e-16,0.047619
tch,3.777301e-16,0.047619
ltg,-3.830854e-16,0.047619
glu,-3.412882e-16,0.047619


* Previamente, estandarizamos las variables y features polinómicos de grado 2 con sus interacciones.

In [11]:
#df = np.array(df)
X = StandardScaler().fit_transform(df)
X = PolynomialFeatures(2,include_bias=True,interaction_only=False).fit_transform(df)
X

PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)


## Paso 1. Hacer la separación entre Train Set y Test Set

Scikit-learn tiene una función que se encarga de la separación entre test y entrenamiento llamada `train_test_split`. El parámetro `test_size` indica la propoción de los datos que vamos a separar para hacer la evaluación.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=53)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(309, 66) (309,)
(133, 66) (133,)


## SOBRE EL TRAIN SET
### Paso 2A. Seleccionar los modelos a evaluar

En este ejercicio probaremos tres modelos diferentes:
    
* regresión lineal (`LinearRegression()`)
* regresión lineal regularizada Ridge (`Ridge()`, `RidgeCV()`)
* regresión lineal regularizada LASSO (`Lasso()`, `LassoCV()`)

Instanciemos, entonces, los modelos.

Ahora bien, vamos a realizar el proceso de tunning de los hiperparámetros de los modelos en base a validación cruzada. Por eso, instanciamos previamente tres objetos:

* un grid de diferentes valores $\alpha$ para realizar el tunning de Ridge (`al_ridge`)
* un grid de diferentes valores $\alpha$ para realizar el tunning de Lasso (`al_lasso`)
* un objeto que genera la partición del dataset en K partes para luego usar en el proceso de validación cruzada (`kf`). Al ejecutar `kf` el mismo provee los índices realizar el split del dataset. Notar que solamente instanciamos el `kf`. Luego, lo utilizaremos en la estimación de los hiperparámetros.

In [6]:
# Generamos un grid de $\alpha$ para probar e instanciamos un particionador del Training Set 
# en K partes para realizar la validación cruzada

al_ridge = np.linspace(0.001, 0.3, 300)
al_lasso = np.linspace(0.1, 0.5, 300)
kf = KFold(n_splits=5, shuffle=True, random_state=12)

# Instanciamos los modelos

lm = LinearRegression()
lmRidgeCV = RidgeCV(alphas=al_ridge, cv=kf, normalize=False)
lmLassoCV = LassoCV(alphas=al_lasso, cv=kf, normalize=False)

* Luego, hacemos el fit de los tres estimadores, lo cual nos lleva a...

### Paso 2B. Proceso de Tunning de Hiperparámetros

In [7]:
# Hacemos los fits respectivos

lm.fit(X_train, y_train)
lmRidgeCV.fit(X_train, y_train)
lmLassoCV.fit(X_train, y_train)

LassoCV(alphas=array([ 0.1    ,  0.10134, ...,  0.49866,  0.5    ]),
    copy_X=True, cv=KFold(n_splits=5, random_state=12, shuffle=True),
    eps=0.001, fit_intercept=True, max_iter=1000, n_alphas=100, n_jobs=1,
    normalize=False, positive=False, precompute='auto', random_state=None,
    selection='cyclic', tol=0.0001, verbose=False)

Es importante tener en claro que sucedió al ejecutar esta celda.

1. Hicimos un fit de un modelo de regresión lineal en el Training Set.
2. Usando `lmRidgeCV`, realizamos el proceso de tunning del $\alpha$ en el modelo regularizado vía Ridge; a su vez, usamos el particionador `kf` con el que definimos que haríamos una Cross-Validation de K=5, haciendo un shuffle del dataset previamente.
3. Usando `lmRidgeCV`, realizamos el tunning del $\alpha$ para el modelo LASSO (usando el mismo particionador `kf`)

Veamos cuáles son los $\alpha$ estimados:

In [8]:
print('Alpha Ridge:',lmRidgeCV.alpha_,'\n'
      'Alpha LASSO:',lmLassoCV.alpha_,'\n')

Alpha Ridge: 0.004 
Alpha LASSO: 0.1 



### Paso 2C. Estimación del modelo final sobre todo el Training Set

Ya estamos en condiciones de realizar la estimación final de los modelos sobre los datos de Training Set, para luego realizar la validación final y seleccionar el modelo a utilizar.

El modelo de regresión lineal (al no tener hiperparámetros) ya está listo. Lo que nos queda es seleccionar los hiperparámetros $\alpha_{ridge}$ y $\alpha_{lasso}$ que minimicen los errores cross-validados respectivos. En este punto, podríamos ejecutar el siguiente código para el caso del modelo Ridge:

``` python

a = lmRidgeCV.alpha_
ridgefinal = Ridge(a)
ridgefinal.fit(X_train, y_train)
```

* Buscamos el $\alpha_{ridge}$ óptimo dentro del objeto `RidgeCV`.
* Instanciamos el estimador `Ridge`.
* Hacemos el fit. 

Con esto, tendríamos estimado el modelo, es decir... todos los parámetros (en este caso, $\beta_{i}$ regularizados. Luego, repetiríamos el mismo proceoso para Lasso

Afortunadamente, Scikit-Learn tiene implementado este procedimiento dentro de los mismos estimadores `RidgeCV` y `LassoCV`. De esta forma, ya se hizo el "fit" con el $\alpha$ mínimo en cada caso. Por esto, podemos llamar directamente al método `.predict` dentro de cada uno de los modelos.

En este [link](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html) y en [este](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html) pueden consultar más en profundidad sobre los estiamdores en cuestión...

**NOTA:** Más adelante en el curso, veremos cómo "encapsular" estos procesos en un proceso más general que se llama [`GridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html), particularmente útil cuando tenemos muchos modelos a evaluar, con muchos hiperparámetros a tunear.

* Solamente por curiosidad, veamos qué performance tiene cada modelo en el Training Set.
* Calculemos el $R^2$ y el MSE.

In [9]:
# Calculamos el R2

print("Score Train Lineal:", lm.score(X_train, y_train),"\n"
      "Score Train Ridge:",  lmRidgeCV.score(X_train, y_train),"\n"
      "Score Train Lasso:",  lmLassoCV.score(X_train, y_train))

# Calculamos el MSE

lmpred_Tr = lm.predict(X_train)
lmRidgepred_Tr = lmRidgeCV.predict(X_train)
lmLassoepred_Tr = lmLassoCV.predict(X_train)

print("Train MSE lineal=", mean_squared_error(y_train,lmpred_Tr), "\n"
      "Train MSE Ridge=",  mean_squared_error(y_train,lmRidgepred_Tr), "\n"
      "Train MSE Lasso=",  mean_squared_error(y_train,lmLassoepred_Tr))

Score Train Lineal: 0.621339560541 
Score Train Ridge: 0.548021177506 
Score Train Lasso: 0.499212866867
Train MSE lineal= 2155.23494961 
Train MSE Ridge= 2572.54377066 
Train MSE Lasso= 2850.34775005


* ¿Cuáles son los parámetros de cada modelo?

In [10]:
(lm.intercept_,lm.coef_)

(85.79953958211587,
 array([  1.74244375e-08,   5.18889669e+01,  -2.77930919e+02,
          3.29161064e+02,   4.08837910e+02,  -1.08791430e+04,
          9.30457503e+03,   4.10294957e+03,   3.98191595e+02,
          4.15989124e+03,   6.36068270e+01,   2.93433860e+03,
          2.60191024e+03,  -2.15843519e+03,   7.48513056e+02,
         -6.84998891e+03,  -2.91786145e+02,   7.40661489e+03,
          5.53513354e+03,   4.59468910e+03,   1.08624636e+01,
         -1.67828092e+00,   2.42996626e+03,   3.54302561e+03,
          6.12788116e+03,  -5.08008608e+03,  -1.08375899e+03,
         -1.35833122e+03,  -2.96844993e+03,   2.15522701e+02,
          1.34378384e+03,   2.35979069e+03,  -1.85731500e+04,
          1.88471882e+04,   2.90855480e+03,  -6.15499109e+03,
          6.08487754e+03,   1.09589190e+03,   1.11728455e+02,
          2.43792531e+04,  -1.33365112e+04,  -1.34970303e+04,
         -8.62545053e+03,  -7.36697671e+03,  -4.09134938e+03,
          4.71639470e+04,  -6.26115020e+04,  -3.20

In [11]:
(lmRidgeCV.intercept_,lmRidgeCV.coef_)

(144.24560945671513,
 array([   0.        ,   25.37551803, -240.58793224,  473.59840184,
         356.98573419, -569.12984278,  341.23098749,  130.94940515,
         287.8208953 ,  611.19149917,   27.37373493,  788.30506737,
         951.32308735,  -73.28252273,  506.98767517,   15.02206887,
        -192.05401658,   66.66432417,   17.36105681,  516.83045087,
         257.76694398,   -1.45278595,  441.84637863,  662.01002896,
         121.67982606,  -90.64170624,  364.67134895, -106.42023249,
          12.28428891,  265.55742485,  590.4685136 ,  424.75897912,
        -109.1358711 ,  -25.85014205,  -60.06010144,   87.10065885,
         108.08628948,  430.00904008,   59.72522251,  301.09163978,
         382.57822162,  -83.07833376,   88.92713352,  198.72533137,
        -149.21851488,  276.64005202,  308.55290108,  390.95314095,
        -401.70422127, -133.87921844,  410.75195998,  245.75637873,
         142.03845794,   23.65881072,   29.1349514 ,  326.06804845,
         -88.09642009,  -77

In [12]:
(lmLassoCV.intercept_,lmLassoCV.coef_)

(151.84474705120286,
 array([   0.        ,    0.        , -159.79396217,  490.65990843,
         305.28197977,   -0.        ,   -0.        , -181.00887023,
          21.98599896,  441.61710179,    4.29887889,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,   -0.        ,    0.        ,    0.        ,
           0.        ,   -0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,   -0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,   -0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
          -0.        ,   -0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
          -0.        ,   -0

* ¿Cuáles son las variables más importantes?

## SOBRE TEST SET
### Paso 3. Hacer la validación final del modelo

Lo único que queda es evaluar la performance del modelo. Para ello debemos usar datos que no hayan formado parte del proceso de entrenamiento anterior. Usaremos, entonces, el Test Set.

In [ ]:
# Hacemos las predicciones sobre la matriz de predictores del Test Set

lmpred_Te = lm.predict(X_test)
lmRidgepred_Te = lmRidgeCV.predict(X_test)
lmLassoepred_Te = lmLassoCV.predict(X_test)

# Testeo final del modelo sobre Test Set

print("Test Score lineal=", mean_squared_error(y_test,lmpred_Te), "\n"
      "Test Score Ridge=",  mean_squared_error(y_test,lmRidgepred_Te), "\n"
      "Test Score Lasso=",  mean_squared_error(y_test,lmLassoepred_Te))

* ¿Con cuál se quedarían? ¿Cuál es el modelo que mejor performa? ¿Y el que peor?